In [ ]:
!pip install spacy
!spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 58.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

cluster = pd.read_csv('/content/drive/MyDrive/24-2 EDA Project/movie_reviews_with_clusters.csv')
review_absa = pd.read_csv('/content/drive/MyDrive/24-2 EDA Project/result_review2.csv')
df_pre = pd.read_csv('/content/drive/MyDrive/24-2 EDA Project/preprocessed_data.csv')

# Replace the address of the applicable dataset with the appropriate one for you

In [ ]:
import ast

df = df_pre.merge(cluster[['userid', 'GMM_Cluster']], on='userid', how = 'left')
df = df.merge(review_absa[['title', 'review1', 'review1_results','review2', 'review2_results']], on = 'title', how = 'left')
df = df.dropna(subset=['review1', 'review2'])
df['review1_results'] = df['review1_results'].apply(ast.literal_eval)
df['review2_results'] = df['review2_results'].apply(ast.literal_eval)

def filter_dict(d):
    return {k: v for k, v in d.items() if v in ['positive', 'negative']}

df['review1_results'] = df['review1_results'].apply(filter_dict)
df['review2_results'] = df['review2_results'].apply(filter_dict)

df = df.drop(['movie_id', 'directedBy', 'starring', 'imdbId'], axis=1).reset_index(drop=True)

In [ ]:
df2 = df.drop_duplicates('title').reset_index(drop=True)

In [ ]:
def replace_parentheses_in_keys(d):
    new_dict = {k.replace('(', '').replace(')', ''): v for k, v in d.items()}
    return new_dict


df2['review1_results'] = df2['review1_results'].apply(replace_parentheses_in_keys)
df2['review2_results'] = df2['review2_results'].apply(replace_parentheses_in_keys)


In [ ]:
def merge_dicts(row):
    review1_dict = row['review1_results']
    review2_dict = row['review2_results']

    combined_dict = {}

    all_keys = set(review1_dict.keys()).union(set(review2_dict.keys()))

    for key in all_keys:
        value1 = review1_dict.get(key)
        value2 = review2_dict.get(key)

        if value1 is not None and value2 is not None:
            if value1 == value2:
                combined_dict[key] = value1
            else:
                combined_dict[key] = 'mixed'
        else:
            combined_dict[key] = value1 if value1 is not None else value2

    return combined_dict


df2['results'] = df2.apply(merge_dicts, axis=1)

In [ ]:
import spacy
import pandas as pd
from tqdm import tqdm

nlp = spacy.load("en_core_web_md")
similarity_threshold = 0.5
tqdm.pandas()

# Identify similarities between reviews with SPACY

def merge_similar_keys(d):
    keys = list(d.keys())
    merged_dict = {}
    merged_indices = set()

    for i in range(len(keys)):
        if i in merged_indices:
            continue
        key1 = keys[i]
        token1 = nlp(key1)

        for j in range(i + 1, len(keys)):
            if j in merged_indices:
                continue
            key2 = keys[j]
            token2 = nlp(key2)
            similarity = token1.similarity(token2)

            if similarity >= similarity_threshold:
                value1 = d[key1]
                value2 = d[key2]
                new_value = 'mixed' if value1 != value2 else value1
                new_key = key1 if len(key1) > len(key2) else key2

                merged_dict[new_key] = new_value
                merged_indices.add(i)
                merged_indices.add(j)
                break
        else:
            merged_dict[key1] = d[key1]

    for i in range(len(keys)):
        if i not in merged_indices:
            merged_dict[keys[i]] = d[keys[i]]

    return merged_dict

df2['results'] = df2['results'].progress_apply(merge_similar_keys)





  0%|          | 8/2528 [00:00<03:28, 12.06it/s]<ipython-input-12-6d5f78cd52b6>:29: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = token1.similarity(token2)
100%|██████████| 2528/2528 [03:41<00:00, 11.42it/s]


In [ ]:
def filter_mixed_entries(d):
    return {k: v for k, v in d.items() if v != 'mixed'}

df2['results'] = df2['results'].apply(filter_mixed_entries)

In [ ]:
def separate_keys(d):
    positive_keys = [k for k, v in d.items() if v == 'positive']
    negative_keys = [k for k, v in d.items() if v == 'negative']
    return pd.Series([positive_keys, negative_keys], index=['positive', 'negative'])

df2[['positive_tag', 'negative_tag']] = df2['results'].apply(separate_keys)



In [ ]:
df = df.merge(df2[['title', 'results', 'positive_tag', 'negative_tag']], on = 'title', how = 'left')

In [ ]:
df.to_csv('/content/drive/MyDrive/24-2 EDA Project/final_df.csv', index=False) #complete the final dataframe